In [1]:
import pandas as pd
import numpy as np

from TREEplus import *

C:\Users\danie\AppData\Local\Temp\ipykernel_21024\1183594070.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
pd.options.display.max_columns = None
dataframe = pd.read_table('lpmc.dat')
print(dataframe.shape)
dataframe.head()

(81086, 32)


,trip_id,household_id,person_n,trip_n,travel_mode,purpose,fueltype,faretype,bus_scale,survey_year,travel_year,travel_month,travel_date,day_of_week,start_time,age,female,driving_license,car_ownership,distance,dur_walking,dur_cycling,dur_pt_access,dur_pt_rail,dur_pt_bus,dur_pt_int,pt_interchanges,dur_driving,cost_transit,cost_driving_fuel,cost_driving_ccharge,driving_traffic_percent
0,0,0,0,0,4,3,1,1,1.0,1,2012,4,1,7,10.000000,56,1,1,2,778,0.218056,0.092222,0.134444,0.0,0.016667,0.0,0,0.052222,1.5,0.14,0.0,0.111702
1,1,0,0,1,4,3,1,1,1.0,1,2012,4,1,7,13.000000,56,1,1,2,778,0.188889,0.061944,0.109444,0.0,0.055556,0.0,0,0.059444,1.5,0.15,0.0,0.112150
2,2,0,0,2,4,3,1,1,1.0,1,2012,4,1,7,17.000000,56,1,1,2,4562,1.037778,0.355833,0.203056,0.0,0.210278,0.0,0,0.236667,1.5,0.79,0.0,0.203052
3,3,0,0,3,4,3,1,1,1.0,1,2012,4,1,7,17.350000,56,1,1,2,4562,1.039444,0.384444,0.205556,0.0,0.258611,0.0,0,0.233333,1.5,0.78,0.0,0.160714
4,4,0,1,2,4,3,1,4,1.0,1,2012,4,1,7,19.583333,54,0,1,2,4562,1.037778,0.355833,0.203056,0.0,0.189444,0.0,0,0.229167,1.5,0.78,0.0,0.130909


In [3]:
#binning age
dataframe = pd.read_table('lpmc.dat')

groupDict={'0-18':0,'19-30':0,'31-65':0, '66+': 0}

for i in dataframe['age']:
    if i>=0 and i<=10:
        groupDict['0-18']+=1
    elif i>=19 and i<=30:
        groupDict['19-30']+=1
    elif i>=31 and i<=65:
        groupDict['31-65']+=1
    else:
        groupDict['66+']+=1

dataframe.loc[dataframe['age']<=18, 'age_group'] = '0-18'
dataframe.loc[dataframe['age'].between(19,30), 'age_group'] = '19-30'
dataframe.loc[dataframe['age'].between(31,65), 'age_group'] = '31-65'
dataframe.loc[dataframe['age']>65, 'age_group'] = '66+'
dataframe.drop(['age'], axis= 1, inplace = True)


#binning distance

dataframe.loc[dataframe['distance']< 658.50, 'distance_group'] = 'distance01'
dataframe.loc[dataframe['distance'].between(658.50,861.49), 'distance_group'] = 'distance02'
dataframe.loc[dataframe['distance'].between(861.50,1214.49), 'distance_group'] = 'distance03'
dataframe.loc[dataframe['distance'].between(1214.50,1514.49 ), 'distance_group'] = 'distance04'
dataframe.loc[dataframe['distance'].between(1514.50, 2156.49), 'distance_group'] = 'distance05'
dataframe.loc[dataframe['distance'].between(2156.50, 2759.49), 'distance_group'] = 'distance06'
dataframe.loc[dataframe['distance'].between(2759.50, 4391.49), 'distance_group'] = 'distance07'
dataframe.loc[dataframe['distance'].between(4391.50, 5369.49), 'distance_group'] = 'distance08'
dataframe.loc[dataframe['distance'].between(5369.50, 10087.49), 'distance_group'] = 'distance09'
dataframe.loc[dataframe['distance'].between(10087.50, 14695.49), 'distance_group'] = 'distance10'
dataframe.loc[dataframe['distance']>=14695.50, 'distance_group'] = 'distance11'
dataframe.drop('distance', axis = 1, inplace = True)

#binning start time

dataframe.loc[dataframe['start_time']<8.091666, 'starttime_group'] = 'early_morning'
dataframe.loc[dataframe['start_time'].between(8.091666,10.00833320), 'starttime_group'] = 'morning'
dataframe.loc[dataframe['start_time'].between(10.008333211,18.50833320), 'starttime_group'] = 'afternoon'
dataframe.loc[dataframe['start_time']> 18.50833321, 'starttime_group'] = 'evening'
dataframe.drop('start_time', axis = 1, inplace = True)

#binning cost transit

dataframe.loc[dataframe['cost_transit']< 1.52499998, 'cost_group'] = 'low_cost'
dataframe.loc[dataframe['cost_transit'].between(1.52499998,3.01999997), 'cost_group'] = 'medium_cost'
dataframe.loc[dataframe['cost_transit']> 3.01999998, 'cost_group'] = 'high_cost'
dataframe.drop('cost_transit', axis = 1, inplace = True)

#binning cost_driving_Fuel

dataframe.loc[dataframe['cost_driving_fuel']< 0.165, 'costdrivingfuel_group'] = 'costdrivingfuelgroup1'
dataframe.loc[dataframe['cost_driving_fuel'].between(0.165,0.204), 'costdrivingfuel_group'] = 'costdrivingfuelgroup2'
dataframe.loc[dataframe['cost_driving_fuel'].between(0.205,0.244), 'costdrivingfuel_group'] = 'costdrivingfuelgroup3'
dataframe.loc[dataframe['cost_driving_fuel'].between(0.245, 0.294), 'costdrivingfuel_group'] = 'costdrivingfuelgroup4'
dataframe.loc[dataframe['cost_driving_fuel'].between(0.295, 0.354), 'costdrivingfuel_group'] = 'costdrivingfuelgroup5'
dataframe.loc[dataframe['cost_driving_fuel'].between(0.355, 0.44499998), 'costdrivingfuel_group'] = 'costdrivingfuelgroup6'
dataframe.loc[dataframe['cost_driving_fuel'].between(0.44499999, 0.83499997), 'costdrivingfuel_group'] = 'costdrivingfuelgroup7'
dataframe.loc[dataframe['cost_driving_fuel'].between(0.83499998, 1.17499994), 'costdrivingfuel_group'] = 'costdrivingfuelgroup8'
dataframe.loc[dataframe['cost_driving_fuel'].between(1.17499995, 2.31499993), 'costdrivingfuel_group'] = 'costdrivingfuelgroup9'
dataframe.loc[dataframe['cost_driving_fuel']> 2.31499994, 'costdrivingfuel_group'] = 'costdrivingfuelgroup10'
dataframe.drop('cost_driving_fuel', axis = 1, inplace = True)

#binning driving traddic percent

dataframe.loc[dataframe['driving_traffic_percent']< 0.24422751, 'trafficperc_group'] = '0low_traffic'
dataframe.loc[dataframe['driving_traffic_percent'].between(0.24422751,0.36449698), 'trafficperc_group'] = '1mediumlow_traffic'
dataframe.loc[dataframe['driving_traffic_percent'].between(0.36449699,0.49620750), 'trafficperc_group'] = '2medium_traffic'
dataframe.loc[dataframe['driving_traffic_percent'].between(0.49620751, 0.57161700), 'trafficperc_group'] = '3mediumhigh_traffic'
dataframe.loc[dataframe['driving_traffic_percent']> 0.57161701, 'trafficperc_group'] = '4high_traffic'
dataframe.drop('driving_traffic_percent', axis = 1, inplace = True)

#binning cost_driving_ccharge

dataframe.loc[dataframe['cost_driving_ccharge']< 5.25, 'drivingcharge_group'] = 'low_charge'
dataframe.loc[dataframe['cost_driving_ccharge']> 5.25, 'drivingcharge_group'] = 'high_charge'
dataframe.drop('cost_driving_ccharge', axis = 1, inplace = True)

In [4]:
for i in dataframe.columns:
      if i not in ['trip_id', 'household_id', 'person_n', 'trip_n', 'travel_date', 'dur_pt_int',
          'dur_pt_rail', 'dur_pt_access', 'dur_pt_bus', 'dur_walking', 'dur_cycling', 'dur_driving']:
            print(i, set(dataframe[i]))

travel_mode {1, 2, 3, 4}
purpose {1, 2, 3, 4, 5}
fueltype {1, 2, 3, 4, 5, 6}
faretype {1, 2, 3, 4, 5}
bus_scale {0.0, 1.0, 0.5}
survey_year {1, 2, 3}
travel_year {2012, 2013, 2014, 2015}
travel_month {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}
day_of_week {1, 2, 3, 4, 5, 6, 7}
female {0, 1}
driving_license {0, 1}
car_ownership {0, 1, 2}
pt_interchanges {0, 1, 2, 3, 4}
age_group {'66+', '0-18', '31-65', '19-30'}
distance_group {'distance09', 'distance03', 'distance07', 'distance06', 'distance08', 'distance01', 'distance05', 'distance10', 'distance11', 'distance02', 'distance04'}
starttime_group {'afternoon', 'early_morning', 'morning', 'evening'}
cost_group {'low_cost', 'medium_cost', 'high_cost'}
costdrivingfuel_group {'costdrivingfuelgroup9', 'costdrivingfuelgroup4', 'costdrivingfuelgroup3', 'costdrivingfuelgroup7', 'costdrivingfuelgroup6', 'costdrivingfuelgroup1', 'costdrivingfuelgroup8', 'costdrivingfuelgroup5', 'costdrivingfuelgroup2', 'costdrivingfuelgroup10'}
trafficperc_group {'2med

In [5]:
y = dataframe[['travel_mode', 'travel_year']]
X = dataframe.drop(['travel_mode', 'survey_year'], axis = 1)

print(set(X["travel_year"]))

a = X.loc[X["travel_year"]==2015]
#creating test set 

X_test= X.query('travel_year == 2015') #creates a subset, like this: X.loc[X["travel_year"]==2015]
y_test = y.query('travel_year == 2015')

X_test = X_test.drop(['travel_year'], axis =1).copy()
y_test = y_test.drop(['travel_year'], axis =1).copy()

#create training set 

X_train_1 = X.query('travel_year == 2012')
y_train_1 = y.query('travel_year == 2012')
X_train_2 = X.query('travel_year == 2013')
y_train_2 = y.query('travel_year == 2013')
X_train_3 = X.query('travel_year == 2014')
y_train_3 = y.query('travel_year == 2014')

X_train = pd.concat([X_train_1, X_train_2, X_train_3])
y_train = pd.concat([y_train_1, y_train_2, y_train_3])

X_train = X_train.drop(['travel_year'], axis = 1)
y_train = y_train.drop(['travel_year'], axis = 1)


data = pd.concat([X_train, y_train], axis = 1).copy()
#data = data_to_shuffle.sample(frac=1, random_state = 42).reset_index(drop=True) #sampling returning all, and resetting index , no seed?
y_train = data['travel_mode']
X_train = data.drop(['travel_mode'], axis = 1)

#feature selection

to_drop = ['trip_id', 'household_id', 'person_n', 'trip_n', 'travel_date', 'dur_pt_int',
          'dur_pt_rail', 'dur_pt_access', 'dur_pt_bus', 'dur_walking', 'dur_cycling', 'dur_driving']
print(f'Number of columns dropped: {len(to_drop)}')
X_train.drop(to_drop, axis = 1, inplace = True)

X_test.drop(to_drop, axis = 1, inplace = True)

predictors = X_test.columns
y = y_test #is the whole vector
print(predictors)

datatest = pd.concat([y_test, X_test], axis = 1).reset_index()
datatest.drop('index', axis = 1, inplace = True)
datatest.to_csv("data_test.csv", index = False)

datatraining = pd.concat([y_train, X_train], axis = 1).reset_index()
datatraining.drop('index', axis = 1, inplace = True)
datatraining.to_csv("data_training.csv", index = False)


{2012, 2013, 2014, 2015}
Number of columns dropped: 12
Index(['purpose', 'fueltype', 'faretype', 'bus_scale', 'travel_month',
       'day_of_week', 'female', 'driving_license', 'car_ownership',
       'pt_interchanges', 'age_group', 'distance_group', 'starttime_group',
       'cost_group', 'costdrivingfuel_group', 'trafficperc_group',
       'drivingcharge_group'],
      dtype='object')


In [6]:
display(y_train.head(10))
print(X_train.shape)
X_train.head(10)

0    4
1    4
2    4
3    4
4    4
5    4
6    4
7    4
8    4
9    4
Name: travel_mode, dtype: int64

(76103, 17)


,purpose,fueltype,faretype,bus_scale,travel_month,day_of_week,female,driving_license,car_ownership,pt_interchanges,age_group,distance_group,starttime_group,cost_group,costdrivingfuel_group,trafficperc_group,drivingcharge_group
0,3,1,1,1.0,4,7,1,1,2,0,31-65,distance02,morning,low_cost,costdrivingfuelgroup1,0low_traffic,low_charge
1,3,1,1,1.0,4,7,1,1,2,0,31-65,distance02,afternoon,low_cost,costdrivingfuelgroup1,0low_traffic,low_charge
2,3,1,1,1.0,4,7,1,1,2,0,31-65,distance08,afternoon,low_cost,costdrivingfuelgroup7,0low_traffic,low_charge
3,3,1,1,1.0,4,7,1,1,2,0,31-65,distance08,afternoon,low_cost,costdrivingfuelgroup7,0low_traffic,low_charge
4,3,1,4,1.0,4,7,0,1,2,0,31-65,distance08,evening,low_cost,costdrivingfuelgroup7,0low_traffic,low_charge
5,3,1,4,1.0,4,7,0,1,2,0,31-65,distance08,evening,low_cost,costdrivingfuelgroup7,0low_traffic,low_charge
6,3,1,3,0.0,4,7,1,0,2,0,0-18,distance02,morning,low_cost,costdrivingfuelgroup1,0low_traffic,low_charge
7,3,1,3,0.0,4,7,1,0,2,0,0-18,distance02,afternoon,low_cost,costdrivingfuelgroup1,0low_traffic,low_charge
8,3,1,3,0.0,4,7,1,0,2,0,0-18,distance08,afternoon,low_cost,costdrivingfuelgroup7,0low_traffic,low_charge
9,3,1,3,0.0,4,7,1,0,2,0,0-18,distance08,evening,low_cost,costdrivingfuelgroup7,0low_traffic,low_charge


In [7]:
X_train.reset_index(inplace = True, drop = True)
y_train.reset_index(inplace = True, drop = True)

X_train=X_train.applymap(str)
#y_train=y_train.astype(str)

n_features = X_train
n_features_names = list(X_train.columns)
indici = np.arange(0, len(y_train))
y = y_train


In [8]:
#need to bin num vars and have features, features_names empty

features = []
features_names = []


negative node prop gain? maybe ok


In [9]:
n_features['purpose'] = 'purpose' + n_features['purpose'].astype(str)
n_features['fueltype'] = 'fueltype' + n_features['fueltype'].astype(str)
n_features['faretype'] = 'faretype' + n_features['faretype'].astype(str)
n_features['bus_scale'] = 'bus_scale' + n_features['bus_scale'].astype(str)
n_features['travel_month'] = 'travel_month' + n_features['travel_month'].astype(str)
n_features['day_of_week'] = 'day_of_week' + n_features['day_of_week'].astype(str)
n_features['female'] = 'female' + n_features['female'].astype(str)
n_features['driving_license'] = 'driving_license' + n_features['driving_license'].astype(str)
n_features['car_ownership'] = 'car_ownership' + n_features['car_ownership'].astype(str)
n_features['pt_interchanges'] = 'pt_interchanges' + n_features['pt_interchanges'].astype(str)
n_features['age_group'] = 'age_group' + n_features['age_group'].astype(str)


n_features.head()

,purpose,fueltype,faretype,bus_scale,travel_month,day_of_week,female,driving_license,car_ownership,pt_interchanges,age_group,distance_group,starttime_group,cost_group,costdrivingfuel_group,trafficperc_group,drivingcharge_group
0,purpose3,fueltype1,faretype1,bus_scale1.0,travel_month4,day_of_week7,female1,driving_license1,car_ownership2,pt_interchanges0,age_group31-65,distance02,morning,low_cost,costdrivingfuelgroup1,0low_traffic,low_charge
1,purpose3,fueltype1,faretype1,bus_scale1.0,travel_month4,day_of_week7,female1,driving_license1,car_ownership2,pt_interchanges0,age_group31-65,distance02,afternoon,low_cost,costdrivingfuelgroup1,0low_traffic,low_charge
2,purpose3,fueltype1,faretype1,bus_scale1.0,travel_month4,day_of_week7,female1,driving_license1,car_ownership2,pt_interchanges0,age_group31-65,distance08,afternoon,low_cost,costdrivingfuelgroup7,0low_traffic,low_charge
3,purpose3,fueltype1,faretype1,bus_scale1.0,travel_month4,day_of_week7,female1,driving_license1,car_ownership2,pt_interchanges0,age_group31-65,distance08,afternoon,low_cost,costdrivingfuelgroup7,0low_traffic,low_charge
4,purpose3,fueltype1,faretype4,bus_scale1.0,travel_month4,day_of_week7,female0,driving_license1,car_ownership2,pt_interchanges0,age_group31-65,distance08,evening,low_cost,costdrivingfuelgroup7,0low_traffic,low_charge


In [10]:
y.value_counts()

travel_mode
4    33595
3    26867
1    13367
2     2274
Name: count, dtype: int64

In [12]:
my_tree = NodeClass('n1', indici) 
tree = TREEplus(y,features,features_names,n_features,n_features_names,impurity_fn = "gini",problem="classifier",method = "LATENT-BUDGET-TREE",twoing = False, max_level =10 , min_cases_parent= 1000,min_cases_child= 500, min_imp_gain = 0.0001) 
tree.growing_tree(my_tree, max_k = 5, combination_split = False, max_c = 5) 


Split Found:  n1 [['car_ownership', ('car_ownership0',), [17746.82661997004, array([6.86946852e-05])], 0]] start
node_proportion_total  0.08692787714564254
Split Found:  n2 [['trafficperc_group', ('0low_traffic', '1mediumlow_traffic', '2medium_traffic'), [15161.326270121412, array([0.00516819])], 1]] left
node_proportion_total  0.4724971836832328
Node_proportion_gain  0.38556930653759025
Split Found:  n4 [['drivingcharge_group', ('high_charge',), [2940.461542516754, array([4.12338039e-31])], 2]] left
node_proportion_total  0.5181924769732803
Node_proportion_gain  0.04569529329004751
Split Found:  n8 [['distance_group', ('distance04', 'distance05', 'distance06', 'distance07', 'distance08', 'distance09', 'distance10', 'distance11'), [1380.2125546568288, array([0.04196496])], 3]] left
node_proportion_total  0.5451034269434386
Node_proportion_gain  0.026910949970158304
Node Search Split Error for:  n16 with obs in node:  624  which is less than minimum parent size:  1000
TypeError: Node se

R[write to console]: Error in solve.default(Tb) : 'a' (1 x 2) must be square



Error in LBA function


R[write to console]: Error in solve.default(Tb) : 'a' (1 x 2) must be square



Error in LBA function


...

R[write to console]: Error in solve.default(Tb) : 'a' (1 x 2) must be square



Error in LBA function


R[write to console]: Error in solve.default(Tb) : 'a' (1 x 2) must be square



Error in LBA function
Node Search Split Error for:  n26 couldn't find an appropiate looking at k variables =  16
TypeError: Node search split (CART) failure
Node Search Split Error for:  n27 with obs in node:  770  which is less than minimum parent size:  1000
TypeError: Node search split (CART) failure
Split Found:  n7 [['cost_group', ('high_cost', 'medium_cost'), [7506.72088857454, array([0.00214913])], 2]] right
node_proportion_total  0.642546750145
Node_proportion_gain  0.00011069046154932316
Split Found:  n14 [['age_group', ('age_group0-18', 'age_group66+'), [3993.3163361261372, array([0.00177856])], 3]] left
node_proportion_total  0.6426466728070439
Node_proportion_gain  9.992266204394973e-05
This split isn't good now i cut it [cat delta]
Split Found:  n15 [['cost_group', ('high_cost',), [3595.1448212268633, array([2.79100869e-31])], 3]] right
node_proportion_total  0.6427405280318944
Node_proportion_gain  9.385522485050313e-05
This split isn't good now i cut it [cat delta]


((((None, (((None, None), ((None, None), (None, None))), (None, None))),
   (None, ((None, None), None))),
  ((((None, (None, None)), None), ((None, None), (None, None))),
   (((None, None), (None, ((None, None), None))),
    (((None, (None, None)), ((((None, None), (None, None)), None), None)),
     (((((None, None), None), (None, (None, None))), (None, (None, None))),
      ((None, None),
       (((None, (None, None)), None),
        (((None, None), (None, None)), (None, None))))))))),
 ((((None, None), (None, (None, None))), (None, None)), (None, None)))

In [13]:
tree.print_tree(table = False, html = False, visual_pruning = True)#.style.hide_index()

In [25]:
tree.print_tree(table = False, html = False, merge_leaves = True, visual_pruning = True)

In [15]:
X_test = X_test.copy()
y_test = y_test.copy()

X_test['purpose'] = 'purpose' + X_test['purpose'].astype(str)
X_test['fueltype'] = 'fueltype' + X_test['fueltype'].astype(str)
X_test['faretype'] = 'faretype' + X_test['faretype'].astype(str)
X_test['bus_scale'] = 'bus_scale' + X_test['bus_scale'].astype(str)
X_test['travel_month'] = 'travel_month' + X_test['travel_month'].astype(str)
X_test['day_of_week'] = 'day_of_week' + X_test['day_of_week'].astype(str)
X_test['female'] = 'female' + X_test['female'].astype(str)
X_test['driving_license'] = 'driving_license' + X_test['driving_license'].astype(str)
X_test['car_ownership'] = 'car_ownership' + X_test['car_ownership'].astype(str)
X_test['pt_interchanges'] = 'pt_interchanges' + X_test['pt_interchanges'].astype(str)
X_test['age_group'] = 'age_group' + X_test['age_group'].astype(str)

X_test.head()

,purpose,fueltype,faretype,bus_scale,travel_month,day_of_week,female,driving_license,car_ownership,pt_interchanges,age_group,distance_group,starttime_group,cost_group,costdrivingfuel_group,trafficperc_group,drivingcharge_group
76103,purpose3,fueltype1,faretype1,bus_scale1.0,travel_month1,day_of_week4,female1,driving_license0,car_ownership1,pt_interchanges1,age_group19-30,distance07,morning,medium_cost,costdrivingfuelgroup7,1mediumlow_traffic,low_charge
76104,purpose3,fueltype1,faretype1,bus_scale1.0,travel_month1,day_of_week4,female1,driving_license0,car_ownership1,pt_interchanges1,age_group19-30,distance07,afternoon,medium_cost,costdrivingfuelgroup7,1mediumlow_traffic,low_charge
76105,purpose3,fueltype1,faretype1,bus_scale1.0,travel_month1,day_of_week4,female0,driving_license1,car_ownership1,pt_interchanges1,age_group31-65,distance07,morning,medium_cost,costdrivingfuelgroup7,1mediumlow_traffic,low_charge
76106,purpose3,fueltype1,faretype1,bus_scale1.0,travel_month1,day_of_week4,female0,driving_license1,car_ownership1,pt_interchanges1,age_group31-65,distance07,afternoon,medium_cost,costdrivingfuelgroup7,1mediumlow_traffic,low_charge
76107,purpose5,fueltype1,faretype1,bus_scale1.0,travel_month1,day_of_week4,female0,driving_license1,car_ownership2,pt_interchanges0,age_group31-65,distance06,afternoon,low_cost,costdrivingfuelgroup7,0low_traffic,low_charge


In [16]:
n_features_test = X_test.copy()
features_test = []

n_features_test.reset_index(inplace = True, drop = True)
y_test.reset_index(inplace = True, drop = True)
y_test_list = y_test["travel_mode"].values.tolist()


pruning not prepared for combination 

In [17]:
y_test['travel_mode'].value_counts()

travel_mode
4    2213
3    1738
1     901
2     131
Name: count, dtype: int64

takes a whle for comb to prune, 100min

In [19]:
alpha, pruned_table = tree.pruning(features_test, n_features_test, y_test_list, table = True)


To return the table, print_tree must be True
Best tree for test set has 22 leaves with misclassification count 1333 


TypeError: 'Styler' object is not callable

In [20]:
pruned_table.style.hide()

Node,Node Type,Variable Split,n,Deviance,Explained Deviance,Class Probabilities,Alpha,Beta,LS Error
n1,Parent,"car_ownership in ('car_ownership0',)",76103,173353.59,0.087,"{1: 0.18, 2: 0.03, 3: 0.35, 4: 0.44}","[[0.89, 0.11], [0.25, 0.75], [0.0, 1.0]]","[[0.28, 0.05, 0.67, 0.0], [0.11, 0.02, 0.17, 0.7]]",[0.0]
n2,Parent,"trafficperc_group in ('0low_traffic', '1mediumlow_traffic', '2medium_traffic')",53924,110112.33,0.472,"{1: 0.14, 2: 0.02, 3: 0.24, 4: 0.59}","[[0.83, 0.17], [0.7, 0.3], [0.54, 0.46], [0.29, 0.71], [0.0, 1.0]]","[[0.17, 0.02, 0.0, 0.81], [0.07, 0.04, 0.64, 0.26]]",[0.01]
n4,Parent,"drivingcharge_group in ('high_charge',)",10509,21215.88,0.518,"{1: 0.09, 2: 0.03, 3: 0.57, 4: 0.31}","[[0.79, 0.21], [0.18, 0.82]]","[[0.08, 0.04, 0.88, 0.0], [0.09, 0.03, 0.36, 0.52]]",[0.0]
n9,Child,None,3944,6073.62,None,"{1: 0.08, 2: 0.04, 3: 0.77, 4: 0.11}","[[0.97, 0.03], [0.91, 0.09], [0.88, 0.12], [0.72, 0.28], [0.0, 1.0]]","[[0.0, 0.04, 0.86, 0.1], [0.3, 0.01, 0.55, 0.14]]",[0.02]
n8,Parent,"distance_group in ('distance04', 'distance05', 'distance06', 'distance07', 'distance08', 'distance09', 'distance10', 'distance11')",6565,13773.49,0.545,"{1: 0.09, 2: 0.03, 3: 0.45, 4: 0.42}","[[0.0, 1.09], [0.22, 0.78], [0.41, 0.59], [0.62, 0.38], [0.72, 0.28], [0.89, 0.11], [0.93, 0.07], [0.97, 0.03], [1.0, 0.0], [1.02, 0.0], [1.03, 0.0]]","[[0.0, 0.04, 0.49, 0.48], [0.78, 0.02, 0.0, 0.24]]",[0.04]
n16,Child,None,624,1183.79,None,"{1: 0.62, 2: 0.02, 3: 0.08, 4: 0.27}",[],[],[]
n17,Parent,"driving_license in ('driving_license0',)",5941,11128.61,0.563,"{1: 0.03, 2: 0.03, 3: 0.49, 4: 0.44}","[[0.63, 0.37], [0.14, 0.86]]","[[0.04, 0.0, 0.76, 0.19], [0.03, 0.05, 0.4, 0.52]]",[0.0]
n35,Child,None,1365,2319.81,None,"{1: 0.04, 2: 0.02, 3: 0.63, 4: 0.32}","[[0.62, 0.38], [0.71, 0.29], [0.32, 0.68], [0.76, 0.24], [0.0, 1.0]]","[[0.03, 0.02, 0.94, 0.0], [0.03, 0.0, 0.41, 0.56]]",[0.01]
n34,Parent,"purpose in ('purpose2', 'purpose3', 'purpose4', 'purpose5')",4576,8662.0,0.575,"{1: 0.03, 2: 0.04, 3: 0.45, 4: 0.48}","[[0.15, 0.85], [1.0, 0.0], [0.76, 0.24], [0.57, 0.43], [0.73, 0.27]]","[[0.06, 0.0, 0.33, 0.61], [0.0, 0.08, 0.6, 0.33]]",[0.0]
n68,Child,None,1481,2755.84,None,"{1: 0.02, 2: 0.06, 3: 0.57, 4: 0.35}",[],[],[]


In [21]:
predictions = []

for i in range(n_features_test.shape[0]):
    d = dict(n_features_test.iloc[i,])
    tree.pred_x(my_tree, d, tree.get_all_node(), tree.get_leaf())
    predictions.append(tree.prediction_cat[-1])


In [22]:
y_test_list = y_test["travel_mode"].values.tolist()

missclass = 0 

for i in range(len(predictions)):
    if predictions[i] != y_test_list[i]:
        missclass += 1 

round(missclass / len(y_test_list) * 100, 2)

26.93

27.61 non combination

26.53 combination

In [23]:
missclass1 = 0 
class1 = 0
missclass2 = 0 
class2 = 0
missclass3 = 0 
class3 = 0
missclass4 = 0 
class4 = 0

for i in range(len(predictions)):
    if y_test_list[i] ==1:
        class1 += 1
        if predictions[i] != y_test_list[i]:
            missclass1 += 1 
    if y_test_list[i] ==2:
        class2 += 1
        if predictions[i] != y_test_list[i]:
            missclass2 += 1 
    if y_test_list[i] ==3:
        class3 += 1
        if predictions[i] != y_test_list[i]:
            missclass3 += 1 
    if y_test_list[i] ==4:
        class4 += 1
        if predictions[i] != y_test_list[i]:
            missclass4 += 1 


"Accuracy", round((1-missclass1 / class1) * 100, 2), round((1-missclass2 / class2) * 100, 2), round((1-missclass3 / class3) * 100, 2), round((1-missclass4 / class4 )* 100, 2)

('Accuracy', 55.6, 0.0, 74.97, 83.01)